# N Puzzle

https://www.hackerrank.com/challenges/n-puzzle

N Puzzle is a sliding blocks game that takes place on a k \* k grid with ((k \* k) - 1) tiles each numbered from 1 to N. Your task is to reposition the tiles to their proper order.


Link to video: https://www.youtube.com/watch?v=XG5ZE11vFvc

Let's solve it with IDA*

## Code

In [1]:
import queue
from math import sqrt

k = 3
blocks = [0, 3, 8, 4, 1, 7, 2, 6, 5]
#k = int(input().strip())
#blocks = [int(input().strip()) for _ in range(k) for _ in range(k)]


class Puzzle(object):
    solution = tuple(range(k*k))     
                                    
    def __init__(self, cells):
        self.cells = tuple(cells)
    
    @property
    def is_solved(self):
        #print(self.solution, self.cells, self.solution == self.cells)
        return self.solution == self.cells

    @property
    def zero_position(self):
        return self.cells.index(0)
    
    def swap(self, index):
        new_cells = list(self.cells)
        new_cells[self.zero_position] = new_cells[index]
        new_cells[index] = 0
        return Puzzle(new_cells)
    
    def manhattan_dist(self, pos1, pos2):
        r1, c1 = pos1//k, pos1%k
        r2, c2 = pos2//k, pos2%k
        return abs(r1 - r2) + abs(c1 - c2)
    
    def distance_to_goal(self):
        positions = zip(self.cells, self.solution)
        return sum([self.manhattan_dist(a, b) for a, b in positions])


class State(object):
    directions = ['UP', 'LEFT', 'RIGHT', 'DOWN']
    
    def __init__(self, puzzle, path=[], states=[]):
        self.puzzle = puzzle
        self.path = path
        self.states = states + [puzzle.cells]
    
    @property
    def is_solved(self):
        return self.puzzle.is_solved

    def branch_toward(self, direction):
        blank_pos = self.puzzle.zero_position
        blank_r = blank_pos // k
        blank_c = blank_pos % k
        cell = None
        if blank_c != 0 and direction == 'LEFT':
            cell = blank_pos - 1
        elif blank_c != k-1 and direction == 'RIGHT':
            cell = blank_pos + 1
        elif blank_r != 0 and direction == 'UP':
            cell = blank_pos - k
        elif blank_r != k-1 and direction == 'DOWN':
            cell = blank_pos + k
        if cell is not None:
            path = self.path + [direction]
            return State(self.puzzle.swap(cell), path, self.states)
    
    @property
    def branches(self):
        adjacents = []
        for d in self.directions:
            adjacent = self.branch_toward(d)
            if adjacent:
                 adjacents.append(adjacent)
        return adjacents
    
    @property
    def cost(self):
        steps_from_start = len(self.path)
        nb_misplaced = sum([i!=j for i,j in zip(self.puzzle.cells, self.puzzle.solution)])
        steps_to_goal = self.puzzle.distance_to_goal()
        return steps_from_start + steps_to_goal + sqrt(nb_misplaced)
        #return sqrt(steps_from_start) + sqrt(steps_to_goal)
        #return nb_misplaced + steps_to_goal

        
def draw_cells(blocks):
    for l in range(k):
        print(' '.join([str(i) for i in blocks[l*k:l*k+k]]))
    print('----')
    

def ida_star(puzzle):
    visited = set()
    frontier = queue.PriorityQueue()
    state = State(puzzle)
    counter = 0
    while not state.is_solved:
        visited.add(state.puzzle.cells)
        for branch in state.branches:
            if branch.puzzle.cells not in visited:
                counter += 1
                frontier.put((branch.cost, counter, branch))
        state = frontier.get()[-1]
    print('Solved in {} moves after finding {} states and navigating through {} of them'.format(
        len(state.path), counter, len(visited)))
    return state


#solution = ida_star(Puzzle(blocks))
#print(len(solution.path))
#print('\n'.join([d for d in solution.path]))

## Animation

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation

def sliding_blocks_animation(solution):
    image = np.zeros((k,k))
    labels = range(k)

    fig, ax =plt.subplots()
    plt.xticks(labels, labels)
    plt.yticks(labels, labels)

    grid = ax.matshow(image, cmap='Pastel1_r')

    def animate(i=0):
        image = np.array(solution.states[i]).reshape((k,k))
        ax.texts = []
        for (i,j), z in np.ndenumerate(image):
            if z != 0:
                ax.text(j, i, z, ha='center', va='center', fontsize=24)
        return (ax.matshow(image, cmap='Pastel1_r'), )

    anim = animation.FuncAnimation(fig, animate, init_func=animate,
                                   frames=len(solution.states), interval=700,
                                   blit=True, repeat=False)
    return anim

In [3]:
import random
from IPython.display import HTML

endgame = range(9)
blocks = list(range(9))
random.shuffle(blocks)

print('Initial state of sliding blocks game:')
draw_cells(blocks)
print('Expected end game')
draw_cells(endgame)

solution = ida_star(Puzzle(blocks))
anim = sliding_blocks_animation(solution)
HTML(anim.to_html5_video())

Initial state of sliding blocks game:
7 4 3
2 0 8
1 6 5
----
Expected end game
0 1 2
3 4 5
6 7 8
----
Solved in 24 moves after finding 3032 states and navigating through 1817 of them
